# SQLAlchemy w/ Core API

The **core** of SQLAlchemy is a set of DB-agnostic, Python constructs that compile and translate to textual SQL. These constructs makeup the __core__ of the SQLAlchemy toolkit and is designed to mimic a lot of the SQL language itself but in Python.

For more see: https://docs.sqlalchemy.org/en/14/core/tutorial.html

In [1]:
from sa2_swing.config import SQLALCHEMY_URI

from sqlalchemy import create_engine

engine = create_engine(SQLALCHEMY_URI, echo=True)

## Create some tables

In [2]:
from sqlalchemy import MetaData, Column, Integer, String, Table

metadata = MetaData()

user_table = Table(
    "user",
    metadata,
    Column('id', Integer, primary_key=True, autoincrement=True),
    Column('first_name', String(30), nullable=False),
    Column('last_name', String(100))
)

metadata.create_all(engine)


2022-04-22 17:44:32,288 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2022-04-22 17:44:32,338 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-04-22 17:44:32,342 INFO sqlalchemy.engine.Engine select current_schema()
2022-04-22 17:44:32,343 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-04-22 17:44:32,345 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2022-04-22 17:44:32,346 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-04-22 17:44:32,350 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-22 17:44:32,352 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2022-04-22 17:44:32,353 INFO sqlalchemy.engine.Engine [generated in 0.00146s] {'name': 'user'}
2022-04-22 17:44:32,357 INFO sqlalchemy.engine.Engine 
CREATE TABLE "user" (
	id SERIAL NOT NULL, 
	first_name VARCHAR(30) NOT NULL, 
	last_name VARCHAR(100), 
	PRIMARY KEY (id)
)


2022-04-22

## Insert some data

In [3]:
from sqlalchemy import insert, text
with engine.connect() as conn:
	conn.execute(text('TRUNCATE TABLE "user"'))

	stmt = insert(user_table).values({"first_name": "Collin", "last_name": "Choy"})
	conn.execute(stmt)

2022-04-22 17:44:32,950 INFO sqlalchemy.engine.Engine TRUNCATE TABLE "user"
2022-04-22 17:44:32,952 INFO sqlalchemy.engine.Engine [generated in 0.00238s] {}
2022-04-22 17:44:32,958 INFO sqlalchemy.engine.Engine COMMIT
2022-04-22 17:44:32,962 INFO sqlalchemy.engine.Engine INSERT INTO "user" (first_name, last_name) VALUES (%(first_name)s, %(last_name)s) RETURNING "user".id
2022-04-22 17:44:32,963 INFO sqlalchemy.engine.Engine [generated in 0.00113s] {'first_name': 'Collin', 'last_name': 'Choy'}
2022-04-22 17:44:32,967 INFO sqlalchemy.engine.Engine COMMIT


In [4]:
with engine.connect() as conn:
	stmt = insert(user_table).values([{"first_name": "Jeff", "last_name": "Bridges"}, {"first_name": "Tim", "last_name": None}])
	conn.execute(stmt)

2022-04-22 17:44:33,789 INFO sqlalchemy.engine.Engine INSERT INTO "user" (first_name, last_name) VALUES (%(first_name_m0)s, %(last_name_m0)s), (%(first_name_m1)s, %(last_name_m1)s)
2022-04-22 17:44:33,793 INFO sqlalchemy.engine.Engine [no key 0.00381s] {'first_name_m0': 'Jeff', 'last_name_m0': 'Bridges', 'first_name_m1': 'Tim', 'last_name_m1': None}
2022-04-22 17:44:33,799 INFO sqlalchemy.engine.Engine COMMIT


## Query some data

In [5]:
from sqlalchemy import select

def select_all_users():
	statement = select(user_table)
	with engine.connect() as conn:
		result = conn.execute(statement)
	return result.fetchall()

select_all_users()

2022-04-22 17:44:35,075 INFO sqlalchemy.engine.Engine SELECT "user".id, "user".first_name, "user".last_name 
FROM "user"
2022-04-22 17:44:35,080 INFO sqlalchemy.engine.Engine [generated in 0.00472s] {}


[(1, 'Collin', 'Choy'), (2, 'Jeff', 'Bridges'), (3, 'Tim', None)]

In [6]:
stmt = select(user_table.c.first_name).where((user_table.c.first_name > "A") & (user_table.c.first_name < "M"))

with engine.connect() as conn:
	result = conn.execute(stmt).all()
result

2022-04-22 17:44:35,565 INFO sqlalchemy.engine.Engine SELECT "user".first_name 
FROM "user" 
WHERE "user".first_name > %(first_name_1)s AND "user".first_name < %(first_name_2)s
2022-04-22 17:44:35,573 INFO sqlalchemy.engine.Engine [generated in 0.00806s] {'first_name_1': 'A', 'first_name_2': 'M'}


[('Collin',), ('Jeff',)]

In [7]:
from sqlalchemy import between

stmt = select(user_table.c.first_name).where(between(user_table.c.first_name, "A", "M"))
with engine.connect() as conn:
	result = conn.execute(stmt).fetchall()
result

2022-04-22 17:44:36,168 INFO sqlalchemy.engine.Engine SELECT "user".first_name 
FROM "user" 
WHERE "user".first_name BETWEEN %(first_name_1)s AND %(first_name_2)s
2022-04-22 17:44:36,172 INFO sqlalchemy.engine.Engine [generated in 0.00400s] {'first_name_1': 'A', 'first_name_2': 'M'}


[('Collin',), ('Jeff',)]

## Update some data

In [8]:
from sqlalchemy import update

stmt = update(user_table).where(user_table.c.first_name == 'Jeff').values(last_name='Goldblum')
with engine.connect() as conn:
	conn.execute(stmt)

select_all_users()


2022-04-22 17:44:37,315 INFO sqlalchemy.engine.Engine UPDATE "user" SET last_name=%(last_name)s WHERE "user".first_name = %(first_name_1)s
2022-04-22 17:44:37,317 INFO sqlalchemy.engine.Engine [generated in 0.00268s] {'last_name': 'Goldblum', 'first_name_1': 'Jeff'}
2022-04-22 17:44:37,325 INFO sqlalchemy.engine.Engine COMMIT
2022-04-22 17:44:37,328 INFO sqlalchemy.engine.Engine SELECT "user".id, "user".first_name, "user".last_name 
FROM "user"
2022-04-22 17:44:37,329 INFO sqlalchemy.engine.Engine [cached since 2.254s ago] {}


[(1, 'Collin', 'Choy'), (3, 'Tim', None), (2, 'Jeff', 'Goldblum')]

## Delete some data

In [9]:
from sqlalchemy import delete

stmt = delete(user_table).where(user_table.c.first_name == "Tim")

with engine.connect() as conn:
	conn.execute(stmt)

select_all_users()

2022-04-22 17:44:38,478 INFO sqlalchemy.engine.Engine DELETE FROM "user" WHERE "user".first_name = %(first_name_1)s
2022-04-22 17:44:38,480 INFO sqlalchemy.engine.Engine [generated in 0.00199s] {'first_name_1': 'Tim'}
2022-04-22 17:44:38,482 INFO sqlalchemy.engine.Engine COMMIT
2022-04-22 17:44:38,485 INFO sqlalchemy.engine.Engine SELECT "user".id, "user".first_name, "user".last_name 
FROM "user"
2022-04-22 17:44:38,486 INFO sqlalchemy.engine.Engine [cached since 3.411s ago] {}


[(1, 'Collin', 'Choy'), (2, 'Jeff', 'Goldblum')]

## Cleanup: Drop tables

In [10]:
metadata.drop_all(engine)

2022-04-22 17:44:39,849 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-22 17:44:39,852 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2022-04-22 17:44:39,854 INFO sqlalchemy.engine.Engine [cached since 7.503s ago] {'name': 'user'}
2022-04-22 17:44:39,860 INFO sqlalchemy.engine.Engine 
DROP TABLE "user"
2022-04-22 17:44:39,863 INFO sqlalchemy.engine.Engine [no key 0.00354s] {}
2022-04-22 17:44:39,868 INFO sqlalchemy.engine.Engine COMMIT
